In [ ]:
# import packages:
import numpy as np                   # Numeric Python
import matplotlib.pyplot as plt      # Plotting routines
import h5py                          # general HDF5 reading/writing library
import rioxarray as rx               # Package to read raster data from hdf5 files
from pyproj import Transformer, CRS  # libraries to allow coordinate transforms
import glob                          # Package to locate files on disk
import os                            # File-level utilities
import re                            # regular expressions for string interpretation
import icepyx as ipx                 # Package to interact with ICESat-2 online resources
from sliderule import icesat2        # Package for online ICESat-2 processing
from scipy import signal
from matplotlib.pyplot import cm
import geopandas

In [ ]:
%matplotlib widget

In [ ]:
plt.close('all')

In [ ]:
# logins, etc.

#HOST = 'https://urs.earthdata.nasa.gov'
#ipx.core.Earthdata.Earthdata('ben_smith','whatever@whatever.io', HOST).login()

url="icesat2sliderule.org"
icesat2.init(url, verbose=False)

In [ ]:
# Annika's bounding box:
# x,y
#-340,-80
#-480,-170

# XR= np.array([-480, -340])*1.e3
# YR= np.array([-170, -80])*1.e3

#North region coordinate limits (in stereographic coords, km): 
region_name = 'hercules-north'
#XR = [-450, -375, -375, -450]
XR = np.array([-450,-375])*1.e3
#YR = [-100, -115, -100, -90]
YR = np.array([-115,-90])*1.e3
#South region coordinate limits (in stereographic coords, km): 
# x = [ -450 -375 -375 -450]; y = [-100 -115 -140 -115];

# shrink down to a tiny box in the center:
XR=np.mean(XR)+np.array([-5.e4, 5.e4])
YR=np.mean(YR)+np.array([-5.e4, 5.e4])

# Prepare coordinate transformations between lat/lon and polar stereographic
crs=CRS.from_epsg(3031)
to_xy_crs=Transformer.from_crs(crs.geodetic_crs, crs)
to_geo_crs=Transformer.from_crs(crs, crs.geodetic_crs)

corners_lat, corners_lon=to_geo_crs.transform(np.array(XR)[[0, 1, 1, 0, 0]], np.array(YR)[[0, 0, 1, 1, 0]])
latlims=[np.min(corners_lat), np.max(corners_lat)]
lonlims=[np.min(corners_lon), np.max(corners_lon)]

In [ ]:
# shp_file = geopandas.read_file('shapes/allan_hills_main_blue_ice_area.shx')
# shp_file.to_file('shapes/allanhills.geojson', driver='GeoJSON')

In [ ]:
#allanhills = icesat2.toregion('shapes/allanhills.geojson',tolerance=0.1,cellsize=0.01)

In [ ]:
# run a slideRule ATL06 query.  Just ask for cycle 8 (Antarctic winter, 2020)
# to avoid getting swamped right away

# See parameters here:
# http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html
params= { 
    'poly':[{'lon':this_lon, 'lat':this_lat} for this_lon, this_lat in zip(corners_lon, corners_lat)],
    #'poly':allanhills['poly'],
        'srt':3,
        'cnf':1,
        'len':10,
         'res':10,
         'ats':5,
         'cnt':10,
         'cycle':8,
         'maxi': 10,
        'pass_invalid':False}

D_IS_SR=icesat2.atl06p(params, 
                    asset="nsidc-s3")

In [ ]:
thistrack = D_IS_SR[(D_IS_SR['rgt']==6)&(D_IS_SR['spot']==3)]
distances = thistrack['distance'].values
heights = thistrack['h_mean'].values
slope = thistrack['dh_fit_dx'].values
scatterfig = plt.figure()
plt.scatter(distances,heights,c=slope)
plt.plot(distances,heights)
plt.show()

In [ ]:
from scipy.fft import fft, fftfreq

yf = fft(heights)
N = len(heights)
step = 10

from scipy.signal import windows# blackman,general_hamming
fftfig = plt.figure()
w = windows.blackman(N)

ywf = fft(heights*w)
xf = fftfreq(N, step)[:N//2]

import matplotlib.pyplot as plt

plt.semilogy(xf[1:N//2], 2.0/N * np.abs(yf[1:N//2]), '-b')
plt.semilogy(xf[1:N//2], 2.0/N * np.abs(ywf[1:N//2]), '-r')
plt.xlim([1e-9,1e-2])

plt.legend(['FFT', 'FFT w. window'])

plt.grid()
plt.show()

In [ ]:
spotset = plt.figure()

n = 50
tracks = D_IS_SR.rgt.unique()
for i in range(1,7):
    try:
        thistrack = D_IS_SR[(D_IS_SR['rgt']==57)&(D_IS_SR['spot']==i)]
        distances = thistrack['distance'].values
        heights = thistrack['h_mean'].values
        heightseg = heights[i*n:(i+1)*n]
        distseg = distances[i*n:(i+1)*n]
        detrend_x = signal.detrend(heightseg)
        plt.plot(np.arange(0,500,10),(detrend_x+(i*.15)),label=('track'+str(i)))
    except:
        pass
plt.legend()

In [ ]:
trackset = plt.figure()
n = 50
tracks = D_IS_SR.rgt.unique()
for i in range(20):
    try:
        thistrack = D_IS_SR[(D_IS_SR['rgt']==(tracks[i]))&(D_IS_SR['spot']==3)]
        distances = thistrack['distance'].values
        heights = thistrack['h_mean'].values
        heightseg = heights[i*n:(i+1)*n]
        distseg = distances[i*n:(i+1)*n]
        detrend_x = signal.detrend(heightseg)
        plt.plot(np.arange(0,500,10),(detrend_x+(i*.15)),label=('track'+str(tracks[i])))
    except:
        pass
plt.legend()
                         

In [ ]:
thistrack = D_IS_SR[(D_IS_SR['rgt']==11)&(D_IS_SR['spot']==3)]
distances = thistrack['distance'].values
heights = thistrack['h_mean'].values
distset = plt.figure()
nlines = 10

for i in range(8):
    heightseg = heights[i*n:(i+1)*n]
    distseg = distances[i*n:(i+1)*n]
    detrend_x = signal.detrend(heightseg)
    plt.plot(np.arange(0,500,10),(detrend_x+(i*.4)),label=('distance'+str(i*n)))
plt.legend()

In [ ]:

npoints = 50
ndist = 5
#tracks = D_IS_SR[D_IS_SR.rgt!=102].rgt.unique()
#ntracks = 20
tracks = [179,128,82,52]
ntracks = 4
ntot = ntracks*ndist
color = cm.viridis(np.linspace(0, 1, ntot))
c_i = 0
curr_y_offset = 0
trackdistset = plt.figure(figsize=[5,ntracks*2])
for track in tracks[:ntracks]:
    try:
        thistrack = D_IS_SR[(D_IS_SR['rgt']==(track))&(D_IS_SR['spot']==3)]
        distances = thistrack['distance'].values
        heights = thistrack['h_mean'].values
        for i in range(ndist):
            heightseg = heights[i*npoints:(i+1)*npoints]
            distseg = distances[i*npoints:(i+1)*npoints]
            detrend_x = signal.detrend(heightseg)
            c = color[c_i]
            c_i = c_i+1
            line_label = 'track '+str(track)+' '+str(i*npoints)+'m'
            trackvar = np.max(detrend_x)-np.min(detrend_x)
            if(trackvar<1):
                curr_y_offset = curr_y_offset+trackvar/2
                plt.plot(np.arange(0,500,10),(detrend_x+(curr_y_offset)),alpha=0.5,c=c,label=line_label)
                plt.text(520,(np.mean(detrend_x)+(curr_y_offset)),line_label)
    except:
         pass
plt.tight_layout()
#plt.legend()